<h1>Research Papers preprocess</h1>
<p>In this notebook, we will convert a folder containing pdf to a .csv files that contains all the paper in it so that it will be easier to do analytics.</p>

In [1]:
# -*- coding: utf-8 -*-
#Import packages
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import timeit
import codecs
import re
import os
from bs4 import BeautifulSoup # $ pip install beautifulsoup4
from wordcloud import WordCloud
%matplotlib inline

In [2]:
#Search through the dataset directory to get all of the file directory.
mypath = '../papers/'
from os import walk
files = []
fileName = []
d = [mypath]
for (dirpath, dirnames, filenames) in walk(mypath):
    if(dirpath)!='./': dirpath+="/"
    dirpath = dirpath.replace("\\","/")
    files.extend([dirpath+s for s in filenames])
    fileName.extend([s for s in filenames])
files[0:10]

['../papers/ICT_professor/Banlue Srisuchinwong/04292722.pdf',
 '../papers/ICT_professor/Banlue Srisuchinwong/05491571.pdf',
 '../papers/ICT_professor/Banlue Srisuchinwong/10.1.1.575.5384.pdf',
 '../papers/ICT_professor/Banlue Srisuchinwong/10.1109@TENCON.2004.1414918.pdf',
 '../papers/ICT_professor/Banlue Srisuchinwong/10438353.pdf',
 '../papers/ICT_professor/Banlue Srisuchinwong/10PAGE72_PAGE77.pdf',
 '../papers/ICT_professor/Banlue Srisuchinwong/18_CMSIM_2013_Munmuangsaen_Srisuchinwong_1_179-185.pdf',
 '../papers/ICT_professor/Banlue Srisuchinwong/2 2014 Post Print.pdf',
 '../papers/ICT_professor/Banlue Srisuchinwong/2 Postprint.pdf',
 '../papers/ICT_professor/Banlue Srisuchinwong/22_CMSIM_2013_Phuphatana_Srisuchinwong_1_209-214.pdf']

In [4]:
import PyPDF2
def convert_pdf_to_txt(path):
    content = ""
    # Load PDF into pyPDF
    pdf = PyPDF2.PdfFileReader(open(path, "rb"))
    # Iterate pages
    for i in range(0, pdf.getNumPages()):
        # Extract text from page and add to content
        content += pdf.getPage(i).extractText() + "\n"
    # Collapse whitespace
    content = " ".join(content.replace("\xa0", " ").strip().split())
    return content

In [ ]:
from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
from pdfminer.converter import TextConverter
from pdfminer.layout import LAParams
from pdfminer.pdfpage import PDFPage
from cStringIO import StringIO
def convert_pdf_to_txt(path):
    rsrcmgr = PDFResourceManager()
    retstr = StringIO()
    codec = 'utf-8'
    laparams = LAParams()
    device = TextConverter(rsrcmgr, retstr, codec=codec, laparams=laparams)
    fp = file(path, 'rb')
    interpreter = PDFPageInterpreter(rsrcmgr, device)
    password = ""
    maxpages = 0
    caching = True
    pagenos=set()

    for page in PDFPage.get_pages(fp, pagenos, maxpages=maxpages, password=password,caching=caching, check_extractable=True):
        interpreter.process_page(page)

    text = retstr.getvalue()

    fp.close()
    device.close()
    retstr.close()
    return text

#Convert Example
#content = convert_pdf_to_txt('papers/Virach Sornlertlamvanich/c7a39b0ff3790651b2297f90f6b0c5bbb632.pdf')

In [5]:
def read_all_pdf(files,verbose=True):
    contents = []
    professor_names = []
    direc = []
    error_counts = 0
    success_counts = 0
    start = timeit.default_timer()
    for file in files:
        professor_name = file.split("/")[-2]
        try:
            content = convert_pdf_to_txt(file)
            contents.append(content)
            professor_names.append(professor_name)
            direc.append(file)
            success_counts+=1
            if(verbose): print("Successfully read "+file+". Moving on..")
        except Exception as e:
            print("error reading "+file+". Moving on..")
            print(e)
            #Remove that pdf
            error_counts+=1
    stop = timeit.default_timer()
    print("Execution time : "+str(round(stop - start,2))+"s.")    
    
    papers_df = pd.DataFrame({'name':professor_names,'content':contents,'directory':direc},columns=['name','content','directory'])
    return papers_df

In [6]:
files[0:5]

['../papers/ICT_professor/Banlue Srisuchinwong/04292722.pdf',
 '../papers/ICT_professor/Banlue Srisuchinwong/05491571.pdf',
 '../papers/ICT_professor/Banlue Srisuchinwong/10.1.1.575.5384.pdf',
 '../papers/ICT_professor/Banlue Srisuchinwong/10.1109@TENCON.2004.1414918.pdf',
 '../papers/ICT_professor/Banlue Srisuchinwong/10438353.pdf']

In [7]:
paper_df = read_all_pdf(files)

error reading ../papers/ICT_professor/Banlue Srisuchinwong/04292722.pdf. Moving on..
EOF marker not found
error reading ../papers/ICT_professor/Banlue Srisuchinwong/05491571.pdf. Moving on..
EOF marker not found


Successfully read ../papers/ICT_professor/Banlue Srisuchinwong/10.1.1.575.5384.pdf. Moving on..
Successfully read ../papers/ICT_professor/Banlue Srisuchinwong/10.1109@TENCON.2004.1414918.pdf. Moving on..
Successfully read ../papers/ICT_professor/Banlue Srisuchinwong/10438353.pdf. Moving on..
Successfully read ../papers/ICT_professor/Banlue Srisuchinwong/10PAGE72_PAGE77.pdf. Moving on..
Successfully read ../papers/ICT_professor/Banlue Srisuchinwong/18_CMSIM_2013_Munmuangsaen_Srisuchinwong_1_179-185.pdf. Moving on..
Successfully read ../papers/ICT_professor/Banlue Srisuchinwong/2 2014 Post Print.pdf. Moving on..
Successfully read ../papers/ICT_professor/Banlue Srisuchinwong/2 Postprint.pdf. Moving on..
Successfully read ../papers/ICT_professor/Banlue Srisuchinwong/22_CMSIM_2013_Phuphatana_Srisuchinwong_1_209-214.pdf. Moving on..
error reading ../papers/ICT_professor/Banlue Srisuchinwong/24_CMSIM_2012_Srisuchinwong_1_273-279.pdf. Moving on..
invalid literal for int() with base 10: b'n'
Su

Successfully read ../papers/ICT_professor/Waree Kongprawechnon/v32_241_251.pdf. Moving on..
Successfully read ../papers/ICT_professor/Waree Kongprawechnon/Vol.17%20No.4%20PART%201.pdf. Moving on..
Successfully read ../papers/ICT_professor/Waree Kongprawechnon/Waree Kongprawechnon_0171BB.pdf. Moving on..
Successfully read ../papers/ICT_professor/Waree Kongprawechnon/Waree Kongprawechnon_81F274.pdf. Moving on..
Successfully read ../papers/ICT_professor/Waree Kongprawechnon/Waree Kongprawechnon_AA8BBF.pdf. Moving on..
Successfully read ../papers/MSME/Bundit Limmeechokchai/0125-3395-32-1-81-89-1.pdf. Moving on..
Successfully read ../papers/MSME/Bundit Limmeechokchai/0c960539b06fe21588000000.pdf. Moving on..
Successfully read ../papers/MSME/Bundit Limmeechokchai/10462534.pdf. Moving on..
Successfully read ../papers/MSME/Bundit Limmeechokchai/10480726.pdf. Moving on..
Successfully read ../papers/MSME/Bundit Limmeechokchai/10480731.pdf. Moving on..
Successfully read ../papers/MSME/Bundit Limm

In [8]:
def isEnglish(s):
    #Could check all character but waste of time.
    return 'e' in s or 'a' in s
cleaned_papers_pdf = paper_df
cleaned_papers_pdf['isValid'] = cleaned_papers_pdf['content'].apply(isEnglish)
cleaned_papers_pdf = cleaned_papers_pdf[cleaned_papers_pdf['isValid']==True]

In [11]:
#Get faculty from file directory
cleaned_papers_pdf.loc[:,'faculty'] = cleaned_papers_pdf['directory'].apply(lambda x : x.split("/")[-3])

A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy [indexing.py:477]


In [15]:
#paper_df.to_csv('..//dataset//paper_content.csv',index=False ,encoding="utf-8")
cleaned_papers_pdf.to_csv('..//dataset//cleaned_papers_pdf.csv',index=False ,encoding="utf-8")